In [4]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from auto_gptq import AutoGPTQForCausalLM

In [6]:


model_id = "./pretrained_models/fffrrt_ruGPT-3.5-13B-GPTQ4"
peft_model_id = './pretrained_models/ai3_gpt3_5_4bit_adapter_epoch17'

model = AutoGPTQForCausalLM.from_quantized(model_id, device="cpu", use_safetensors=True)
model.load_adapter(peft_model_id)

The safetensors archive passed at ./pretrained_models/fffrrt_ruGPT-3.5-13B-GPTQ4/gptq_model-4bit-128g.safetensors does not contain metadata. Make sure to save your model with the `save_pretrained` method. Defaulting to 'pt' metadata.
GPT2GPTQForCausalLM hasn't fused attention module yet, will skip inject fused attention.
GPT2GPTQForCausalLM hasn't fused mlp module yet, will skip inject fused mlp.


AssertionError: 